#Librairie 

In [128]:
# !pip install pydantic openpyxl
# !pip install pypdf
# !pip install -qU langchain-community langchain-openai
# !pip install pdf2image Pillow
# !pip install pdfplumber
# !pip install tqdm
# !pip install unstructured
# !pip install matplotlib
# #dbutils.library.restartPython()

In [13]:
import pandas as pd
import os
import json
from dataclasses import dataclass
import base64
import requests
from typing import Dict, List, Optional, Any, Union
from PIL import Image
import matplotlib.pyplot as plt
from openpyxl.utils import get_column_letter
import openpyxl
from matplotlib.table import Table
from pathlib import Path
from tqdm import tqdm


#Init SecureGPT

In [14]:

@dataclass
class AuthConfig:
    """Configuration for authentication"""
    base_url: str
    client_id: str
    client_secret: str
    scope: str

class AuthClient:
    """Client for handling authentication"""
    def __init__(self, config: AuthConfig):
        self.config = config

    def get_token(self) -> Dict[str, str]:
        """Get OAuth2 access token"""
        try:
            response = requests.post(
                f"{self.config.base_url}/as/token.oauth2",
                headers={"Content-Type": "application/x-www-form-urlencoded"},
                data={
                    "grant_type": "client_credentials",
                    "client_id": self.config.client_id,
                    "client_secret": self.config.client_secret,
                    "scope": self.config.scope
                }
            )
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            raise Exception(f"Authentication failed: {str(e)}")


In [15]:
def get_auth_token() -> str:
    """Helper function to get authentication token"""
    config = AuthConfig(
        base_url="https://onelogin.stg.axa.com",
        client_id="VJKkswZb",
        client_secret="yG3pl6nkYhvJlQVD1y7xbZBl692YRytdpUJeODKg5BAutoEgxhkqzXLFYGqH5zlp",
        scope="urn:grp:chatgpt"
    )
    client = AuthClient(config)
    result = client.get_token()
    return result.get("access_token", "")

class SecureGPTChat:
    """Main class for interacting with SecureGPT API"""
    def __init__(self, 
                token: str,
                temperature: float = 0.05,
                base_url: str = "https://api-pp.se.axa-go.axa.com/ago-m365-securegpt-bapi-v1-vrs",
                model: str = "gpt-4o-2024-08-06"):
        self.token = token
        self.temperature = temperature
        self.base_url = base_url
        self.model = model
        self.costs = {
            'input_token': 2.50/1_000_000,
            'output_token': 10.00/1_000_000,
            'base_image_cost': 0.001913
        }

    def calculate_cost(self, response: Dict, image_path: Optional[str] = None) -> Dict[str, float]:
        """Calculate cost of API usage"""
        total_cost = 0
        metrics = {'token_cost': 0, 'image_cost': 0}

        if 'usage' in response:
            input_tokens = response['usage'].get('prompt_tokens', 0)
            output_tokens = response['usage'].get('completion_tokens', 0)
            metrics['token_cost'] = (input_tokens * self.costs['input_token'] + 
                                   output_tokens * self.costs['output_token'])
            total_cost += metrics['token_cost']

        if image_path:
            try:
                with Image.open(image_path) as img:
                    width, height = img.size
                    pixels = width * height
                    metrics['image_cost'] = (pixels / (1024 * 1024)) * self.costs['base_image_cost']
                    total_cost += metrics['image_cost']
            except Exception as e:
                print(f"Error calculating image cost: {str(e)}")

        metrics['total_cost'] = total_cost
        return metrics

    def encode_image(self, image_path: str) -> str:
        """Encode image to base64"""
        if image_path.startswith('data:image'):
            return image_path.split('base64,')[1]
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    def prepare_messages(self, messages: List[Dict[str, Any]], 
                        image_path: Optional[str] = None) -> List[Dict]:
        """Prepare messages for API request"""
        formatted_messages = []
        for msg in messages:
            content = msg["content"]
            if image_path and msg == messages[-1]:
                content = [
                    {
                        "type": "image_url",
                        "image_url": f"data:image/jpeg;base64,{self.encode_image(image_path)}",
                        "detail": "auto"
                    },
                    {"type": "text", "text": content}
                ]
            formatted_messages.append({
                "role": msg.get("role", "user"),
                "content": content
            })
        return formatted_messages

    def chat(self, messages: List[Dict[str, Any]], 
             image_path: Optional[str] = None, 
             **kwargs) -> Dict:
        """Send chat request to API"""
        url = f"{self.base_url}/deployments/{self.model}/chat/completions"
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.token}"
        }
        data = {
            "messages": self.prepare_messages(messages, image_path),
            "temperature": self.temperature,
            "max_tokens": kwargs.get("max_tokens", 12000)
        }

        try:
            response = requests.post(url, headers=headers, json=data)
            response.raise_for_status()
            api_response = response.json()
            cost_metrics = self.calculate_cost(api_response, image_path)
            api_response['cost_metrics'] = cost_metrics
            return api_response
        except requests.exceptions.RequestException as e:
            print(f"API Error Details: {response.text if 'response' in locals() else 'No response'}")
            raise Exception(f"API call failed: {str(e)}")

In [16]:
def create_chat_interface():
    """Create an interactive chat interface with the model"""
    token = get_auth_token()
    model = SecureGPTChat(
        temperature=0.0,
        model="gpt-4o-mini-2024-07-18",
        token=token
    )
    history = []
    
    def chat_with_model():
        while True:
            user_input = input("\nVous: ")
            if user_input.lower() in ['quit', 'exit', 'bye']:
                print("\nAu revoir!")
                break
                
            # Add message to history
            history.append({"role": "user", "content": user_input})
            
            # Get model response
            try:
                response = model.chat(history)
                bot_response = response['choices'][0]['message']['content']
                history.append({"role": "assistant", "content": bot_response})
                print("\nAssistant:", bot_response)
            except Exception as e:
                print(f"\nErreur: {str(e)}")
                continue
    
    return chat_with_model


##Image ton Json 

In [17]:
def extract_structure_with_securegpt(model: SecureGPTChat, 
                                  image_path: str, 
                                  example_json: Dict) -> Dict:
   messages = [
       {
           "role": "system",
           "content": f"""Tu es un expert en assurance analysant des tableaux Excel de paramètres d'endoso.

Structure requise:
0. S'il y a pas de colonne enfin si tu vois qu'il y a comme colonne standard parameters ce n'est pas une colonne mais c'est juste pour dire que les variables à côté et en dessous sont de type "std" donc quand il n'y a pas de colonne tu mets juste column_parameters : '' en gros un vide. S'il y a quelque chose qui ressemble à une garantie là tu mets sinon rien et surtout tu ne mets pas standard parameters...
1. Les paramètres sous "Standard Parameters" doivent avoir type="std"
1bis. Les paramètres sous "Parameters in exception" doivent avoir type="exception"
1ter. Idem pour les options il peut y avoir d'autres cas que tu verras forcément sur l'image en dessous de standard, exception etc..
2. Les paramètres doivent être regroupés par "column_parameters" si des colonnes sont visibles
3. Si pas de colonnes visibles, utiliser uniquement "raw_parameters" à remplir et column_parameters : ""

Voici un exemple de la structure attendue:
{json.dumps(example_json, indent=2)}

Points importants:
- Créer les column_parameters uniquement si des colonnes sont visibles dans le tableau
- Standard/Exception définissent uniquement le type du paramètre, pas la structure, il n'apparaissent pas dans la structure
- Garder la même structure que l'exemple fourni
- Respecter les noms exacts des champs présents dans l'exemple
- Copier les valeurs textuelles exactes du tableau : TRES IMPORTANT C'EST A DIRE MEME S'IL Y A UN TEXTE LONG AVEC PLUSIEURS PHRASES DANS UNE CELLULE TU NE METS PAS QUE "TEXT" TU RECOPIES TOUT LE TEXTE.

Retourne uniquement le JSON."""
       },
       {
           "role": "user", 
           "content": "Extrais la structure JSON en respectant les règles prédéfinies."
       }
   ]
   
   response = model.chat(messages, image_path=image_path)
   return json.loads(response['choices'][0]['message']['content'].strip().replace('```json', '').replace('```', ''))

#Partie II extraction des garanties / param

In [18]:
def transform_results_to_dataframe(json_path: str) -> pd.DataFrame:
    """Transform JSON results into a pandas DataFrame"""
    try:
        # Load JSON data
        with open(json_path, 'r', encoding='utf-8') as f:
            results = json.load(f)
            
        # Initialize lists for transformed data
        transformed_data = []
        
        for result in results:
            row_data = {
                'endoso_code': result.get('endoso_code'),
                'original_text': result.get('text'),
                'cost_input_tokens': result.get('cost_metrics', {}).get('token_cost', 0)
            }
            
            # Extract parameters if they exist
            params = result.get('extracted_parameters', {}).get('parameters', [])
            if isinstance(params, list):
                for param in params:
                    prefix = f"{param.get('row_parameter_name', 'unknown')}_"
                    row_data[prefix + 'value'] = param.get('value')
                    row_data[prefix + 'unit'] = param.get('unit')
                    row_data[prefix + 'type'] = param.get('type')
            
            transformed_data.append(row_data)
        
        # Create DataFrame
        df = pd.DataFrame(transformed_data)
        
        # Organize columns
        fixed_cols = ['endoso_code', 'original_text', 'cost_input_tokens']
        param_cols = [col for col in df.columns if col not in fixed_cols]
        
        return df[fixed_cols + sorted(param_cols)]
        
    except Exception as e:
        print(f"Error transforming results to DataFrame: {str(e)}")
        return pd.DataFrame()

def transform_to_flat_dataframe(json_path: str) -> pd.DataFrame:
    """Transform JSON results into a flat DataFrame structure"""
    def flatten_dict(d: Dict, parent_key: str = '', sep: str = '.') -> Dict:
        items = []
        for k, v in d.items():
            new_key = f"{parent_key}{sep}{k}" if parent_key else k
            
            if isinstance(v, dict):
                items.extend(flatten_dict(v, new_key, sep=sep).items())
            elif isinstance(v, list) and v and isinstance(v[0], dict):
                for i, item in enumerate(v):
                    if isinstance(item, dict):
                        items.extend(flatten_dict(item, f"{new_key}_{i}", sep=sep).items())
            else:
                items.append((new_key, v))
        return dict(items)
    
    try:
        # Load JSON
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # Flatten each record
        flattened_data = [flatten_dict(item) for item in data]
        
        # Convert to DataFrame
        df = pd.DataFrame(flattened_data)
        
        # Keep only columns with actual values
        value_counts = df.count()
        useful_cols = value_counts[value_counts > 0].index.tolist()
        
        return df[useful_cols]
        
    except Exception as e:
        print(f"Error transforming to flat DataFrame: {str(e)}")
        return pd.DataFrame()

In [19]:
class EndosoParameterExtractor:
    """Class to handle endosso parameter extraction"""
    def __init__(self, model: SecureGPTChat, structure_dir: str = "json_structures"):
        self.model = model
        self.structure_dir = structure_dir
        self.structures_cache = {}
        
        # Load examples at initialization
        try:
            with open('input_endoso_example.txt', 'r') as f:
                self.input_example = f.read()
            with open('output_endoso_example.json', 'r') as f:
                self.output_example = json.load(f)
        except Exception as e:
            print(f"Error loading examples: {str(e)}")
            self.input_example = ""
            self.output_example = {}
    
    def load_structure(self, endoso_code: str) -> Dict:
        """Load JSON structure for a given endoso code"""
        if endoso_code in self.structures_cache:
            return self.structures_cache[endoso_code]
        
        json_path = Path(self.structure_dir) / f"{endoso_code}.json"
        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                structure = json.load(f)
                self.structures_cache[endoso_code] = structure
                return structure
        except Exception as e:
            print(f"Error loading structure for {endoso_code}: {str(e)}")
            return {}
    
    def extract_parameters(self, text: str, structure: Dict, endoso_code: str, contract_no: str, subgrupo: str) -> Dict:
        """Extract parameters from text using the provided structure"""
        conversation = [
            {
                "role": "system", 
                "content": "Tu es un extracteur intelligent de garanties d'endosos. Tu sortiras UNIQUEMENT LE JSON PAS DE PHRASE A COTE QUAND JE TE LE DEMANDERAI UNIQUEMENT LE JSON"
            },
            {
                "role": "user",
                "content": "Je vais te montrer comment extraire les garanties."
            },
            {
                "role": "user",
                "content": f"Input exemple:\n{self.input_example}"
            },
            {
                "role": "user",
                "content": f"Output exemple:\n{json.dumps(self.output_example, indent=2)}"
            },
            {
                "role": "assistant",
                "content": "Compris, je vais adapter l'extraction au format demandé en fonction de l'endoso avec tous les paramètres comme l'input exemple vers l'output exemple."
            },
            {
                "role": "user",
                "content": f"""Voici la structure de l'endoso, il faudra bien respecter la structure et les données de l'endoso le texte que je te fournis :\n{json.dumps(structure, indent=2)}
                
                Le json fournit donne uniquement la structure et les modalités possible par contre TRES IMPORTANT SUIS SURTOUT L'EXEMPLE FOURNI PLUS HAUT EN RESPECTANT le data model du json

                De plus il faudra (c'est super important) que tu respectes EXACTEMENT la structuer que je t'aie fourni :
                    - LA STRUCTURE TE DONNE LES ELEMENTS IMPORTANT POUR REMPLIR LA STRUCTURE : par exemple y/n sont les modalités définissant que ça peut être Y ou N, pareil digits ça veut dire que c'est une valeur numérique etc...
                    - TU RETOURNERAS TOUJOURS LA MEME STRUCTURE TU N'AJOUTERAS PAS DE PARAMETRES, RIEN ! ET SI TU NE TROUVES PAS DE VALEUR TU METTRAS SIMPLEMENT UN VIDE : "" ou NA si c'est numérique.
                    - TU N'INVENTERAS PAS DE STRUCTURE SUPPLEMENTAIRE : PAS DE COLONNE EN PLUS SI TU NE LA VOIS PAS DANS LA STRUCTURE OU SUBCOLUMN OU ROW OU SUBROW OU ROW PARAMETER ETC... TU RESPECTERAS LA STRUCTURE.clear
                    - POUR DESCRIPTION TU RECOPIERAS EXACTEMENT LA DESCRIPTION TU NE FERAS NI MODIFICATION NI INTERPRETATION : MOT POUR MOT
                
                Maintenant qu'on a nos règles : 

                Utilise cette structure et le texte suivant pour générer le JSON final:\n{text}

                Tu transformeras donc le texte en json structurée en t'inspirant des exemples et en appliquant bien évidemment la structure demandé complété des paramètres demandés qui se trouvent dans le texte, en gros les deux avec toutes les informations sois exhaustives. Rappel : tu suis la structure fourni, tu trouves les valeurs que dans le texte et tu remplis certaines données dans le json via les données ci dessous

                Tu mettras toujours les valeurs dans value s'il y en a pas tu mets ''
                                
                Toujours dans un format Json. Jamais de données superflus après pas de phrases rien. 

                l'endoso_master_id ici est : {endoso_code.replace('endoso_','')}, 
                l'endoso est : {endoso_code},
                le numéro de contrat est : {contract_no},
                le subgroupe est : {subgrupo}

                Tu ajouteras toujours ces informations endoso id, numéro de contrat et subgroupe ... toujours.

                Tu retourneras toujours un json correspondant au json de paramètre transmis."""
            }
        ]
        
        try:
            response = self.model.chat(conversation)
            json_str = response['choices'][0]['message']['content'].strip()
            # Clean JSON string
            json_str = json_str.replace('```json', '').replace('```', '').strip()
            return {
                'parameters': json.loads(json_str),
                'cost_metrics': response.get('cost_metrics', {})
            }
        except Exception as e:
            print(f"Error extracting parameters: {str(e)}")
            return {'parameters': {}, 'cost_metrics': {}}

def process_dataframe(df: pd.DataFrame, model: SecureGPTChat, output_file: str = 'results.json'):
    """Process entire DataFrame of endossos"""
    results = []
    # Initialiser le modèle
    
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing endossos"):
        model = SecureGPTChat(
                temperature=0.0,
                model="gpt-4o-2024-08-06",  # Utilisation du modèle le plus récent
                token=get_auth_token()
            )
        extractor = EndosoParameterExtractor(model)
        
        try:
            endoso_code = row['endoso_code']
            contract_no = row['POLIZA']
            subgrupo = row['NOM_SUBG']
            text_content = row['CONT']
            
            # Load structure for endoso
            structure = extractor.load_structure(endoso_code)
            if not structure:
                print(f"Structure not found for {endoso_code}")
                continue
            
            # Extract parameters
            result = extractor.extract_parameters(text_content, structure, endoso_code,contract_no,subgrupo)
            
            # Store result
            results.append({
                'endoso_code': endoso_code,
                'text': text_content,
                'extracted_parameters': result['parameters'],
                'cost_metrics': result['cost_metrics'],
                'nom_subg':subgrupo,
                'contract_no':contract_no
            })
            
        except Exception as e:
            print(f"Error processing row: {str(e)}")
            continue
    
    # Save results
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    
    return results

In [20]:
# Initialiser le modèle
model = SecureGPTChat(
    temperature=0.0,
    model="gpt-4o-2024-08-06",  # Utilisation du modèle le plus récent
    token=get_auth_token()
)

###Brouillon test sur l'extraction 

In [21]:
# Initialiser le modèle
model = SecureGPTChat(
    temperature=0.0,
    model="gpt-4o-2024-08-06",  # Utilisation du modèle le plus récent
    token=get_auth_token()
)

# Lecture et préparation des données
print("\nChargement des données...")
df = pd.read_csv('Endosos_Final.CSV', sep=',', encoding='ISO-8859-1')

with open('mapping_endosoName_code.json', 'r') as f:
    mapping = json.load(f)

def get_endoso_code(description: str, mapping: list) -> str:
    description_lower = description.lower().strip()
    try:
        for item in mapping:
            for key in item.keys():
                if key.lower() in description_lower or description_lower in key.lower():
                    return item[key]
        return ''
    except:
        return ''

df['endoso_code'] = df['DRIPCION'].fillna('').apply(lambda x: get_endoso_code(x, mapping))
print(f"Données chargées: {len(df)} lignes")


Chargement des données...
Données chargées: 2113803 lignes


In [93]:
df.sample(2)

,TIPO_ENDOSO,POLIZA,SUBRAMO,RFC,NOMBRE,FEC_INI,FEC_VEN,FEC_EMI,NUMSUB,NOM_SUBG,CVE_LIT,DRIPCION,CONT,endoso_code
927804,LIBRE_CAPTURA,FW41816E,GMC,ASM060912MQ0,"AGGREKO SERVICES MEXICO, S.A. DE C.V",20190101,20200101,20181210,NaN,General,11.0,REAN-006 RECONOCIMIENTO DE ANT,REAN-006 RECONOCIMIENTO DE ANTIGÜEDAD PARA RED...,
1649652,LIBRE_CAPTURA,FW57922A,GMC,CMA791101EU9,"LAS CERVEZAS MODELO DEL ALTIPLANO, S",20171231,20181231,20180125,0.0,CONFIDE,4.0,INIC-048 INICIO DE COBERTURA D,_La Compañía cubrirá los gastos médicos erogad...,


In [29]:
import pandas as pd
import re

def normalize_text(text):

    if not isinstance(text, str):
        return text  # Retourne le texte tel quel s'il ne s'agit pas d'une chaîne
    
    # Mettre en minuscules
    text = text.lower()
    
    # Supprimer la ponctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Enlever les espaces en début et fin de chaîne
    text = text.strip()
    
    # Remplacer les espaces multiples par un seul espace
    text = re.sub(r'\s+', ' ', text)
    
    return text
df_dedup = df.drop_duplicates(['CONT']).copy()
df_dedup['CONT_NORMALIZED'] = df_dedup['CONT'].apply(normalize_text)

In [30]:
df_dedup = df_dedup.drop_duplicates(['CONT_NORMALIZED'])

In [31]:
df_dedup.shape

(136266, 15)

In [24]:
temp = df_dedup.groupby('endoso_code',as_index=False).size().sort_values('size')
temp['cumsum'] = temp['size'].cumsum()
temp = temp[temp['cumsum']<1000]
temp

,endoso_code,size,cumsum
15,endoso_97,2,2
3,endoso_132,3,5
1,endoso_122,5,10
2,endoso_127,5,15
5,endoso_150,6,21
7,endoso_155,51,72
13,endoso_82,121,193
4,endoso_142,132,325
9,endoso_38,135,460
14,endoso_84,429,889


In [ ]:
df_dedup.head(1)

In [70]:
endoso_code_list_1 = ["endoso_97",
"endoso_132",
"endoso_127",
"endoso_150",
"endoso_155",
"endoso_82"] 

endoso_code_list = ['endoso_84','endoso_38','endoso_142',"endoso_97","endoso_132","endoso_127",
                    "endoso_150","endoso_155","endoso_82"]
df_temp = df_dedup[df_dedup['endoso_code'].isin(temp['endoso_code'])]
df_temp = df_dedup[df_dedup['endoso_code'].isin(endoso_code_list)]
df_temp.shape

(884, 15)

In [71]:
df[df.endoso_code.isin(endoso_code_list)].POLIZA.nunique()

36366

In [73]:
# df_temp = df_temp.sample(10)
df_temp.shape

(884, 15)

In [78]:
import pandas as pd
import os

# Supposons que df_temp est votre DataFrame d'origine
# Exemple de création d'un DataFrame pour le démonstrateur
# df_temp = pd.DataFrame(...)

# Définition de la taille de l'échantillon
sample_size = 50

# Traitement des endossos
print("\nTraitement des endossos...")
results = []
for i in range(0, len(df_temp), sample_size):
    # Découper le DataFrame en échantillons de 50 lignes
    df_sample = df_temp.iloc[i:i + sample_size]
    
    # Définir les indices pour le nom du fichier
    start_index = i + 1
    end_index = min(i + sample_size, len(df_temp))
    file_name = f'resultats_endossos_{start_index}_{end_index}.csv'
    
    # Vérifier si le fichier existe déjà
    if os.path.exists(file_name):
        print(f"Le fichier '{file_name}' existe déjà. Passage au prochain échantillon.")
        continue  # Passer au prochain échantillon si le fichier existe
    
    # Traiter l'échantillon
    processed_results = process_dataframe(df_sample, model)
    results.append(processed_results)
    
    # Sauvegarde des résultats dans un CSV
    df_results = pd.DataFrame(processed_results)
    df_results.to_csv(file_name, index=False, encoding='utf-8-sig')
    
    print(f"Traitement terminé: {len(processed_results)} endossos traités pour l'échantillon {start_index}_{end_index}")
    print(f"Résultats sauvegardés dans '{file_name}'")

print("\nTous les échantillons ont été traités et sauvegardés.")



Traitement des endossos...
Le fichier 'resultats_endossos_1_50.csv' existe déjà. Passage au prochain échantillon.
Le fichier 'resultats_endossos_51_100.csv' existe déjà. Passage au prochain échantillon.
Le fichier 'resultats_endossos_101_150.csv' existe déjà. Passage au prochain échantillon.
Le fichier 'resultats_endossos_151_200.csv' existe déjà. Passage au prochain échantillon.
Le fichier 'resultats_endossos_201_250.csv' existe déjà. Passage au prochain échantillon.
Le fichier 'resultats_endossos_251_300.csv' existe déjà. Passage au prochain échantillon.
Le fichier 'resultats_endossos_301_350.csv' existe déjà. Passage au prochain échantillon.
Le fichier 'resultats_endossos_351_400.csv' existe déjà. Passage au prochain échantillon.
Le fichier 'resultats_endossos_401_450.csv' existe déjà. Passage au prochain échantillon.
Le fichier 'resultats_endossos_451_500.csv' existe déjà. Passage au prochain échantillon.
Le fichier 'resultats_endossos_501_550.csv' existe déjà. Passage au prochain

In [61]:
tempo = pd.read_csv('resultats_endossos_1_50.csv',encoding='utf-8-sig')

In [62]:
import ast 
def string_to_dict(s):
    return ast.literal_eval(s.replace("'", '"'))

# Appliquer la fonction pour créer une nouvelle colonne avec des dictionnaires
tempo['cost_dict'] = tempo['cost_metrics'].apply(string_to_dict)

# Extraire les valeurs de 'total_cost' et calculer la somme
total_sum = tempo['cost_dict'].apply(lambda x: x['total_cost']).sum()

print("La somme totale des 'total_cost' est :", total_sum)

La somme totale des 'total_cost' est : 1.0127975000000002


In [129]:
import pandas as pd
import json
import os

def json_to_df_single_row(row):
    # Vérification du contenu de 'extracted_parameters'
    extracted_parameters_str = row['extracted_parameters']
    
    # Remplacer les simples guillemets par des doubles guillemets
    extracted_parameters_str = extracted_parameters_str.replace("'", '"')
    
    # Chargement du JSON
    try:
        extracted_parameters = json.loads(extracted_parameters_str)  # Charger le JSON
    except json.JSONDecodeError as e:
        print(f"Erreur de décodage JSON pour la ligne: {e}")
        print(f"Contenu de extracted_parameters: {extracted_parameters_str}")  # Afficher le contenu problématique
        return pd.DataFrame()  # Retourner un DataFrame vide en cas d'erreur
    
    endoso_details = extracted_parameters['endoso_details']
    
    # Données de base de l'endoso
    base_data = {
        'endoso_id': row['endoso_code'],
        'name': endoso_details['name'],
        'type': extracted_parameters['type'],
        'contract_no': extracted_parameters['contract_no'],
        'original_text': row.get('text', '')
    }
    
    # Liste pour stocker les résultats
    rows = []
    
    # Traiter les paramètres de colonnes
    if endoso_details['parameters'].get('column_parameters'):
        for col in endoso_details['parameters']['column_parameters']:
            for detail in col['column_parameter_details']:
                for param in detail.get('row_parameters', []):
                    row_data = base_data.copy()
                    row_data.update({
                        'column': col['column_parameter_name'],
                        'sub_column': detail.get('sub_column_parameter_name', ''),
                        'parameter': param['row_parameter_name'],
                        'parameter_type': param.get('type', ''),
                        'value_type': param.get('value_type', ''),
                        'value': param.get('value', ''),
                        'unit': param.get('unit', '')
                    })
                    rows.append(row_data)
    
    # Traiter les paramètres de lignes
    if endoso_details['parameters'].get('row_parameters'):
        for param in endoso_details['parameters']['row_parameters']:
            row_data = base_data.copy()
            row_data.update({
                'column': '',
                'sub_column': '',
                'parameter': param['row_parameter_name'],
                'parameter_type': param.get('type', ''),
                'value_type': param.get('value_type', ''),
                'value': param.get('value', ''),
                'unit': param.get('unit', '')
            })
            rows.append(row_data)
    
    return pd.DataFrame(rows)

# Chargement du fichier CSV et conversion en DataFrame
random_file = list(filter(lambda x: 'resultats_endossos_' in x, os.listdir()))
random_index = 5
random_file = random_file[random_index] if random_index < len(random_file) else random_file[0]
resultats_df = pd.read_csv(random_file)

# Choisir une seule ligne (par exemple, la première ligne)
single_row = resultats_df.iloc[0]  # Remplacez 0 par l'index de la ligne que vous voulez traiter

# Appliquer la fonction pour obtenir le DataFrame "aplati" pour cette ligne
temporaire = json_to_df_single_row(single_row)

# Affichage des résultats
temporaire

# Optionnel : exportation vers Excel
# temporaire.to_excel('endosos_flattened_single.xlsx', index=False)


,endoso_id,name,type,contract_no,original_text,column,sub_column,parameter,parameter_type,value_type,value,unit
0,endoso_84,URGENCIA MÉDICA EN EL EXTRANJERO (UMEX),standard,FW79371B,"UMEX-032 URGENCIA MÉDICA EN EL EXTRANJERO __""L...",Urgencias Médicas,,suma asegurada,std,,"100,000",USD
1,endoso_84,URGENCIA MÉDICA EN EL EXTRANJERO (UMEX),standard,FW79371B,"UMEX-032 URGENCIA MÉDICA EN EL EXTRANJERO __""L...",Urgencias Médicas,,deducible,std,,100,USD
2,endoso_84,URGENCIA MÉDICA EN EL EXTRANJERO (UMEX),standard,FW79371B,"UMEX-032 URGENCIA MÉDICA EN EL EXTRANJERO __""L...",Urgencias Médicas,,coaseguro,std,,0,%
3,endoso_84,URGENCIA MÉDICA EN EL EXTRANJERO (UMEX),standard,FW79371B,"UMEX-032 URGENCIA MÉDICA EN EL EXTRANJERO __""L...",Urgencias Médicas,,información adicional,std,,,
4,endoso_84,URGENCIA MÉDICA EN EL EXTRANJERO (UMEX),standard,FW79371B,"UMEX-032 URGENCIA MÉDICA EN EL EXTRANJERO __""L...",Urgencias Médicas,,gastos cubiertos,std,,,
5,endoso_84,URGENCIA MÉDICA EN EL EXTRANJERO (UMEX),standard,FW79371B,"UMEX-032 URGENCIA MÉDICA EN EL EXTRANJERO __""L...",Urgencias Médicas,,gastos no cubiertos,std,,,
6,endoso_84,URGENCIA MÉDICA EN EL EXTRANJERO (UMEX),standard,FW79371B,"UMEX-032 URGENCIA MÉDICA EN EL EXTRANJERO __""L...",Urgencias Médicas,,aplicación de deducible y/o coaseguro,std,,false,
7,endoso_84,URGENCIA MÉDICA EN EL EXTRANJERO (UMEX),standard,FW79371B,"UMEX-032 URGENCIA MÉDICA EN EL EXTRANJERO __""L...",Urgencias Médicas,,condiciones generales,std,,"Para los demás términos y condiciones, aplicar...",
8,endoso_84,URGENCIA MÉDICA EN EL EXTRANJERO (UMEX),standard,FW79371B,"UMEX-032 URGENCIA MÉDICA EN EL EXTRANJERO __""L...",Urgencias Médicas,,elegibilidad,exception,,,
9,endoso_84,URGENCIA MÉDICA EN EL EXTRANJERO (UMEX),standard,FW79371B,"UMEX-032 URGENCIA MÉDICA EN EL EXTRANJERO __""L...",Urgencias Médicas,,periodo de espera,exception,,,


In [130]:
# import pandas as pd
# import json

# def json_to_df(data):
#     # Listes pour stocker les données
#     rows = []
    
#     # Traiter le JSON
#     endoso_details = data['endoso_details']
    
#     # Données de base de l'endoso
#     base_data = {
#         'endoso_id': data['endoso_id'],
#         'endoso_master_id': data['endoso_master_id'],
#         'contract_no': data['contract_no'],
#         'subgrupo': data['subgrupo'],
#         'type': data['type'],
#         'original_text': data.get('text', ''),
#         'legal': endoso_details.get('legal', ''),
#         'global_information': data['global_information'],
#         'business_rules': data['business_rules'],
#         'status': data['status'],
#         'comment': data['comment']
#     }
    
#     # Traiter les paramètres de colonnes
#     if endoso_details['parameters'].get('column_parameters'):
#         for col in endoso_details['parameters']['column_parameters']:
#             for detail in col['column_parameter_details']:
#                 for param in detail.get('row_parameters', []):
#                     row = base_data.copy()
#                     row.update({
#                         'column': col['column_parameter_name'],
#                         'sub_column': detail.get('sub_column_parameter_name', ''),
#                         'parameter': param['row_parameter_name'],
#                         'parameter_type': param.get('type', ''),
#                         'value_type': param.get('value_type', ''),
#                         'value': param.get('value', ''),
#                         'unit': param.get('unit', '')
#                     })
#                     rows.append(row)
    
#     # Traiter les paramètres de lignes
#     if endoso_details['parameters'].get('row_parameters'):
#         for param in endoso_details['parameters']['row_parameters']:
#             row = base_data.copy()
#             row.update({
#                 'column': '',
#                 'sub_column': '',
#                 'parameter': param['row_parameter_name'],
#                 'parameter_type': param.get('type', ''),
#                 'value_type': param.get('value_type', ''),
#                 'value': param.get('value', ''),
#                 'unit': param.get('unit', '')
#             })
#             rows.append(row)
    
#     return pd.DataFrame(rows)

# def get_json_example(filepath):
#     with open(filepath, 'r', encoding='utf-8') as json_file:
#         json_content = json_file.read()
#         json_data = json.loads(json_content)
#     return json_data

# # Exemple d'utilisation
# list_files_json = os.listdir('resultats_json')

# random_index = 3  # Choisir un index valide
# data = get_json_example('resultats_json/' + list_files_json[random_index])
# temporaire = json_to_df(data)

# # Exportation en Excel si besoin
# # temporaire.to_excel('endosos_flattened.xlsx', index=False)
# temporaire.drop(['global_information','business_rules'],axis=1)

In [131]:
#  for endoso in data:
#         endoso_details = endoso['extracted_parameters']['endoso_details']
        
#         # Données de base de l'endoso
#         base_data = {
#             'endoso_id': endoso['endoso_code'],
#             'name': endoso_details['name'],
#             'type': endoso['extracted_parameters']['type'],
#             'contract_no': endoso['extracted_parameters']['contract_no'],
#             'original_text': endoso.get('text', '')
#         }
        
#         # Traiter les paramètres de colonnes
#         if endoso_details['parameters'].get('column_parameters'):
#             for col in endoso_details['parameters']['column_parameters'
# data

In [132]:
# import pandas as pd
# import os
# import json

# # Créez un sous-dossier pour sauvegarder les fichiers JSON
# output_dir = 'resultats_json'
# os.makedirs(output_dir, exist_ok=True)

# # Parcourez tous les fichiers dans le répertoire courant
# for filename in os.listdir():
#     if filename.endswith('.csv') and 'resultats_' in filename:
#         print(f"Traitement du fichier: {filename}")
        
#         # Lire le fichier CSV
#         df_temp = pd.read_csv(filename)

#         # Parcourir les lignes du DataFrame
#         for index, row in df_temp.iterrows():
#             # Extraire les JSON des colonnes spécifiées
#             json_columns = ['extracted_parameters']  # Ajoutez d'autres colonnes si nécessaire

#             for col in json_columns:
#                 if col in row:
#                     json_data = row[col]

#                     # Convertir la chaîne JSON en dictionnaire
#                     try:
#                         json_object = eval(json_data)  # Utiliser eval pour évaluer les chaînes de dictionnaire
#                     except (SyntaxError, NameError) as e:
#                         print(f"Erreur de format dans la colonne '{col}' à la ligne {index} du fichier '{filename}': {e}")
#                         continue  # Passer à la ligne suivante en cas d'erreur

#                     # Définir le nom du fichier à partir de l'index et de la colonne
#                     file_name = os.path.join(output_dir, f'{os.path.splitext(filename)[0]}_row_{index + 1}_{col}.json')

#                     # Sauvegarder le JSON dans un fichier
#                     with open(file_name, 'w', encoding='utf-8') as json_file:
#                         json.dump(json_object, json_file, ensure_ascii=False, indent=4)

#                     # print(f"Fichier JSON sauvegardé: '{file_name}'")

# print("\nTous les fichiers JSON ont été sauvegardés.")

In [146]:
# for element in os.listdir('resultats_json/'):
#     with element 



# Exemple d'utilisation
# json_str = '{"endoso_id": "endoso_132", "endoso_details": {"parameters": {"column_parameters": "", "row_parameters": [{"row_parameter_name": "Descripcion", "value": ""}]}}}'
# data = json.loads(json_str)

# normalized_data = normalize_json(data)
# print(json.dumps(normalized_data, indent=4))


In [154]:
import pandas as pd
import json
import os
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Font

def clean_json_string(json_str):
    # Remplacez les chaînes vides par des listes vides ou des objets vides
    json_str = json_str.replace('"column_parameters": "",', '"column_parameters": [],')
    json_str = json_str.replace('"legal": "",', '"legal": {},')
    return json_str

def normalize_json(data):
    if isinstance(data, dict):
        for key, value in data.items():
            if value == "":
                data[key] = []  # Remplacer les chaînes vides par des listes vides
            elif isinstance(value, list):
                data[key] = [normalize_json(item) for item in value]
            else:
                data[key] = normalize_json(value)
    elif isinstance(data, list):
        for index, item in enumerate(data):
            data[index] = normalize_json(item)
    return data

def json_to_df_single_row(row):
    extracted_parameters_str = row['extracted_parameters'].replace("'", '"')
    extracted_parameters_str = clean_json_string(extracted_parameters_str)
    
    try:
        extracted_parameters = json.loads(extracted_parameters_str)
    except json.JSONDecodeError as e:
        print(f"Erreur de décodage JSON pour la ligne: {e}")
        print(f"Contenu de extracted_parameters: {extracted_parameters_str}")
        return pd.DataFrame()
    
    extracted_parameters = normalize_json(extracted_parameters)
    endoso_details = extracted_parameters['endoso_details']
    
    base_data = {
        'endoso_id': row['endoso_code'],
        'name': endoso_details['name'],
        'type': extracted_parameters['type'],
        'contract_no': extracted_parameters['contract_no'],
        'original_text': row.get('text', '')
    }
    
    rows = []
    
    if endoso_details['parameters'].get('column_parameters'):
        for col in endoso_details['parameters']['column_parameters']:
            for detail in col.get('column_parameter_details', []):
                for param in detail.get('row_parameters', []):
                    row_data = base_data.copy()
                    row_data.update({
                        'column': col.get('column_parameter_name', ''),
                        'sub_column': detail.get('sub_column_parameter_name', ''),
                        'parameter': param['row_parameter_name'],
                        'parameter_type': param.get('type', ''),
                        'value_type': param.get('value_type', ''),
                        'value': param.get('value', ''),
                        'unit': param.get('unit', '')
                    })
                    rows.append(row_data)
    
    if endoso_details['parameters'].get('row_parameters'):
        for param in endoso_details['parameters']['row_parameters']:
            row_data = base_data.copy()
            row_data.update({
                'column': '',
                'sub_column': '',
                'parameter': param['row_parameter_name'],
                'parameter_type': param.get('type', ''),
                'value_type': param.get('value_type', ''),
                'value': param.get('value', ''),
                'unit': param.get('unit', '')
            })
            rows.append(row_data)
    
    return pd.DataFrame(rows)

csv_files = list(filter(lambda x: 'resultats_endossos_' in x, os.listdir()))
dataframes = []

for file in csv_files:
    df = pd.read_csv(file)
    dataframes.append(df)

resultats_df = pd.concat(dataframes, ignore_index=True)
unique_endosos = resultats_df.drop_duplicates(subset=['endoso_code'])

with pd.ExcelWriter('endosos_summary.xlsx', engine='openpyxl') as writer:
    for index, row in unique_endosos.iterrows():
        parameters_df = json_to_df_single_row(row)
        
        if parameters_df.empty:
            continue
        
        sheet_name = row['endoso_code'][:31]
        worksheet = writer.book.create_sheet(sheet_name)
        
        endoso_text = row['text']
        worksheet.cell(row=1, column=1, value=endoso_text).font = Font(bold=True, size=14)
        worksheet.cell(row=1, column=1).fill = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')

        parameters_df.to_excel(writer, sheet_name=sheet_name, startrow=2, startcol=0, index=False)

        header_format = Font(bold=True, color='FFFFFF')
        fill_header = PatternFill(start_color='0000FF', end_color='0000FF', fill_type='solid')
        for col in range(1, parameters_df.shape[1] + 1):
            worksheet.cell(row=2, column=col).font = header_format
            worksheet.cell(row=2, column=col).fill = fill_header

print("Fichier Excel créé avec succès.")


Erreur de décodage JSON pour la ligne: Expecting value: line 1 column 570 (char 569)
Contenu de extracted_parameters: {"endoso_id": "endoso_155", "endoso_master_id": "155", "contract_no": "FW98876A", "subgrupo": "SUBGRUPO 1", "type": "standard", "endoso_details": {"name": "Factor", "key": "", "title": "", "subtitle": "", "description": "El Factor de Reembolso de cada Procedimiento Terapéutico incluido en las Condiciones Generales correspondientes a esta póliza esta expresado de manera porcentual, por lo que para cualquier efecto dicho factor deberá dividirse entre 100.", "parameters": {"column_parameters": [], "row_parameters": [{"row_parameter_name": "Descripcion", "editable": True, "active": True, "type": "std", "value_type": "text", "value": "El Factor de Reembolso de cada Procedimiento Terapéutico incluido en las Condiciones Generales correspondientes a esta póliza esta expresado de manera porcentual, por lo que para cualquier efecto dicho factor deberá dividirse entre 100."}, {"ro